Just a reminder:
RCFD2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Consolidated Bank). Dollar amounts in thousands.
RCON2107: Obligations (other than securities and leases) of states and political subdivisions in the
U.S (Domestic Offices). Dollar amounts in thousands.

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import io
import os
import gc

directory = os.fsencode('data')
muni_banks_table = pd.DataFrame()
quarter = {'0331': 'Q1', '0630': 'Q2', '0930': 'Q3', '1231': 'Q4'}

for file in os.listdir(directory):
    # prep the zipfiles to open
    filename = os.fsdecode(file)
    zipf = zipfile.ZipFile('data/'+filename)
    
    # find some designation to find the right file:
    words = filename.split()
    call_report_date = words[-1].replace('.zip', '')
    
    # find the bank designation
    bank_filename = 'FFIEC CDR Call Bulk POR {}.txt'.format(call_report_date)
    bank_file = pd.read_csv(zipf.open(bank_filename), sep='\t').set_index('IDRSSD')

    # grab the call schedule
    securities_filename = 'FFIEC CDR Call Schedule RCCI {}.txt'.format(call_report_date)
    securities_file = pd.read_csv(zipf.open(securities_filename), sep='\t', header=[0], skiprows=[1]).set_index('IDRSSD')

    # just suss out the muni data
    muni_loans = securities_file['RCFD2107'].dropna().to_frame()
    muni_loans['CallDate'] = call_report_date[-4:]+quarter[call_report_date[0:4]]
    
    # merge the tables together, append to dataframe
    muni_bank = pd.merge(muni_loans, bank_file, how='left', left_index=True, right_index=True)
    if muni_banks_table.empty:
        muni_banks_table = muni_bank
    else:
        muni_banks_table = pd.concat([muni_banks_table, muni_bank])
        
    # garbage clean up
    del [[bank_file, securities_file, muni_loans, muni_bank]]
    gc.collect()
    bank_file = pd.DataFrame()
    securities_file = pd.DataFrame()
    muni_loans = pd.DataFrame()
    muni_bank = pd.DataFrame()

In [ ]:
# let's look at the sums
muni_banks_table[muni_banks_table['RCFD2107'] != 0]
muni_total = muni_banks_table.groupby(['CallDate', 'Financial Institution Name'])['RCFD2107'].sum().unstack(level=-2).fillna(0)
temp = muni_total.transpose()
temp['ALL BANKS'] = temp.sum(axis=1)
muni_total = temp.transpose()
muni_total = muni_total/1000
muni_total['sum'] = muni_total.sum(axis=1)
muni_total = muni_total[muni_total['sum'] != 0]
muni_total.drop('sum', axis=1, inplace=True)

total_only = muni_total[muni_total.index=='ALL BANKS'].copy().transpose()
# let's drop the total for now DELETE THIS LINE IF YOU WANT TO KEEP IT
muni_total = muni_total.drop('ALL BANKS', axis=0)

Then taking all that work and converting it into an easy to read csv for heroku to fly with.

In [ ]:
muni_total.to_csv('munidata.csv')
total_only.to_csv('munitotal.csv')

In [3]:
muni_total

CallDate,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,2002Q3,2002Q4,2003Q1,2003Q2,...,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2
Financial Institution Name,,,,,,,,,,,,,,,,,,,,,
1ST FINANCIAL BANK USA,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.487,0.394,0.366,0.366,0.336,0.301,0.259,0.000,0.000,0.000
ALLFIRST BANK,149.013,131.908,132.427,183.958,149.131,108.758,94.130,81.253,86.016,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
ALLY BANK,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,46.000,64.000,67.000,68.000,79.000
AMALGAMATED BANK,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"AMEGY BANK, NATIONAL ASSOCIATION",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AMERICAN EXPRESS CENTURION BANK,0.012,0.016,0.012,0.011,0.021,0.022,0.016,0.013,0.017,0.014,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AMERICAN NATIONAL BANK AND TRUST COMPANY OF CHICAGO,147.942,144.968,131.878,140.272,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
AMSOUTH BANK,116.404,106.037,120.367,123.661,119.420,112.245,123.335,78.360,70.434,62.848,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
ARVEST BANK,0.060,0.060,0.060,0.045,0.045,8.343,7.971,10.075,10.890,11.025,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
